In [1]:
import cv2
import numpy as np
import time
from os import walk
import pickle

In [2]:
net = cv2.dnn.readNet("yolov3-tiny.weights","yolov3-tiny.cfg") #Tiny Yolo
classes = ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [3]:
imageFileNames = []
for (dirpath, dirnames, filenames) in walk("imageDir"):
    imageFileNames.extend(filenames)
    break

In [4]:
layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors= np.random.uniform(0,255,size=(len(classes),3))
output = {}
for fileName in imageFileNames:
    imageFile = 'imageDir/' + fileName
    img = cv2.imread(imageFile)
    img = cv2.resize(img, (320,320))
    font = cv2.FONT_HERSHEY_PLAIN
    height,width,channels = img.shape
    blob = cv2.dnn.blobFromImage(img,(1.0/255.0),(320,320),(0,0,0),True,crop=False) 
    net.setInput(blob)
    outs = net.forward(outputlayers)
    class_ids=[]
    confidences=[]
    boxes=[]
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                #object detected
                center_x= int(detection[0]*width)
                center_y= int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x=int(center_x - w/2)
                y=int(center_y - h/2)

                boxes.append([x,y,w,h]) #put all rectangle areas
                confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
                class_ids.append(class_id) #name of the object tha was detected
    output[fileName] = (class_ids, confidences, boxes)
output

{'image1.jpg': ([2, 2],
  [0.953926146030426, 0.9795964360237122],
  [[184, 138, 147, 62], [186, 135, 145, 71]]),
 'image10.jpg': ([2, 2],
  [0.950482964515686, 0.9820160269737244],
  [[61, 134, 149, 65], [69, 133, 139, 72]]),
 'image100.jpg': ([2, 2],
  [0.8476512432098389, 0.7054803371429443],
  [[86, 138, 157, 58], [89, 133, 158, 69]]),
 'image1000.jpg': ([], [], []),
 'image1001.jpg': ([], [], []),
 'image1002.jpg': ([], [], []),
 'image1003.jpg': ([], [], []),
 'image1004.jpg': ([], [], []),
 'image1005.jpg': ([], [], []),
 'image1006.jpg': ([], [], []),
 'image1007.jpg': ([], [], []),
 'image1008.jpg': ([], [], []),
 'image1009.jpg': ([], [], []),
 'image101.jpg': ([2, 2],
  [0.9078163504600525, 0.8571061491966248],
  [[76, 141, 155, 56], [68, 135, 165, 69]]),
 'image1010.jpg': ([], [], []),
 'image1011.jpg': ([], [], []),
 'image1012.jpg': ([], [], []),
 'image1013.jpg': ([], [], []),
 'image1014.jpg': ([], [], []),
 'image1015.jpg': ([], [], []),
 'image1016.jpg': ([], [], []),

In [5]:
pickle.dump(output,open("TinyYolo.pkl",'wb'))